Running Tensorboard from Jupyter lab
====================================
---
Introduction au Deep Learning  (IDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  
Vesion : 1.0

## 1/ Méthode 1 : Shell execute
### 1.1/ Start/stop

In [2]:
%%bash
tensorboard_start --logdir ./run/logs

Tensorbord started - pid is  9934
12399


In [7]:
%%bash
tensorboard_status

Tensorboard status - not found...


In [3]:
%%bash
tensorboard_stop

Tensorbord stopped - pid was 9934


### 1.3/ Scripts

In [2]:
%%writefile "~/bin/tensorboard_start"

#!/bin/bash
#
# -----------------------------------------------------------
#  _____                         _                         _ 
# |_   _|__ _ __  ___  ___  _ __| |__   ___   __ _ _ __ __| |
#   | |/ _ \ '_ \/ __|/ _ \| '__| '_ \ / _ \ / _` | '__/ _` |
#   | |  __/ | | \__ \ (_) | |  | |_) | (_) | (_| | | | (_| |
#   |_|\___|_| |_|___/\___/|_|  |_.__/ \___/ \__,_|_|  \__,_|
#                                                       Start
# -----------------------------------------------------------
# Start tensorboard, with calculate port number and good host
# -----------------------------------------------------------
# Jean-Luc Parouty CNRS/SIMaP Janvier 2020 - version 1.02

VERSION='1.2'
CONDA_ENV='GPU'
HOST_FRONT='f-dahu'

# ---- Usage
#
if [ "$1" == "-?" ] || [ $# -eq 0 ]; then
        echo -e "Start tensorboard in GRICAD environment. (v$VERSION)"
        echo -e "Usage: $(basename $0) -h | --logdir <logdir> [tensorboard args]"
        echo -e "Exemple :  $(basename $0) --logdir ./run/logs\n"
        exit
fi
                        
# ---- Conda activate
#
source /applis/environments/conda.sh
conda activate "$CONDA_ENV" >/dev/null 2>&1
status=$?
if [ $status -ne 0 ]; then
        echo -e "Oups ! Conda environment is not available... contact your support ;-(\n"
        exit 1
fi
                        
# ---- Port number
#
PORT_JPY="$(/usr/bin/id -u)"
PORT_TSB="$(( $PORT_JPY + 10000 ))"
                        
# ---- tmpdir (tmp bug)
#
export TMPDIR="/tmp/$(/usr/bin/id -un)"
/bin/mkdir -p "$TMPDIR"

# ---- Start it
#
tensorboard --port $PORT_TSB --host 0.0.0.0 $@  &>/dev/null &
                     
# ---- Where is it ?
#
sleep 5
p="$(/bin/ps ax | /bin/grep "tensorboard --port $PORT_TSB" | /bin/grep -v grep | awk '{print $1}')"
if [ -z "$p" ]; then
        echo "Tensorboard didn't start... check your parameters !"
        exit
else
        echo "Tensorbord started - pid is  $p"
fi
                        
# ---- Not on a node ? we give the tunnel
#
if [ "$(hostname)" == $HOST_FRONT ]
then
    SSH_CMD="/usr/bin/ssh -NL 6006:$HOST_FRONT:$PORT_TSB dahu.ciment"
    echo -e "SSH Tunnel : \e[93m$SSH_CMD \e[0m\n"
fi
   

Overwriting /home/paroutyj/bin/tensorboard_start


In [ ]:
%%writefile "~/bin/tensorboard_status"

#!/bin/bash
#
# -----------------------------------------------------------
#  _____                         _                         _ 
# |_   _|__ _ __  ___  ___  _ __| |__   ___   __ _ _ __ __| |
#   | |/ _ \ '_ \/ __|/ _ \| '__| '_ \ / _ \ / _` | '__/ _` |
#   | |  __/ | | \__ \ (_) | |  | |_) | (_) | (_| | | | (_| |
#   |_|\___|_| |_|___/\___/|_|  |_.__/ \___/ \__,_|_|  \__,_|
#                                                      Status
# -----------------------------------------------------------
# Stop tensorboard previously started with tensorboard_start
# -----------------------------------------------------------
# Jean-Luc Parouty CNRS/SIMaP Janvier 2020

VERSION="1.02"

# ---- Usage
#
if [ "$1" == "-?" ] ; then
        echo -e "Tensorboard status in GRICAD environment. (v$VERSION)"
        echo -e "Usage: $(basename $0) [-h ]"
        echo -e "Exemple :  $(basename $0)\n"
        exit
fi

# ---- Process id
#
PORT_JPY="$(id -u)"
PORT_TSB="$(( $PORT_JPY + 10000 ))"

# ---- Where is it ?
#
p="$(ps ax | grep "tensorboard --port $PORT_TSB" | grep -v grep | awk '{print $1}')"
if [ -z "$p" ]; then
        echo "Tensorboard status - not found..."
else
        echo "Tensorboard status - pid is  $p"
fi

In [ ]:
%%writefile "~/bin/tensorboard_stop"

#!/bin/bash
#
# -----------------------------------------------------------
#  _____                         _                         _ 
# |_   _|__ _ __  ___  ___  _ __| |__   ___   __ _ _ __ __| |
#   | |/ _ \ '_ \/ __|/ _ \| '__| '_ \ / _ \ / _` | '__/ _` |
#   | |  __/ | | \__ \ (_) | |  | |_) | (_) | (_| | | | (_| |
#   |_|\___|_| |_|___/\___/|_|  |_.__/ \___/ \__,_|_|  \__,_|
#                                                       Stop
# -----------------------------------------------------------
# Stop tensorboard previously started with tensorboard_start
# -----------------------------------------------------------
# Jean-Luc Parouty CNRS/SIMaP Janvier 2020

VERSION="1.02"

# ---- Usage
#
if [ "$1" == "-?" ] ; then
        echo -e "Stop tensorboard in GRICAD environment. (v$VERSION)"
        echo -e "Usage: $(basename $0) [-h ]"
        echo -e "Exemple :  $(basename $0)\n"
        exit
fi

# ---- Process id
#
PORT_JPY="$(id -u)"
PORT_TSB="$(( $PORT_JPY + 10000 ))"

# ---- Where is it ?
#
p="$(ps ax | grep "tensorboard --port $PORT_TSB" | grep -v grep | awk '{print $1}')"
if [ -z "$p" ]; then
        echo "Tensorboard process not found..."
else
        kill $p
        echo "Tensorbord stopped - pid was $p"
fi

**Set scripts privileges :**

In [ ]:
%%bash
/bin/chmod 755 ~/bin/tensorboard_*  2>/dev/null
/bin/ls -l ~/bin/tensorboard_*      2>/dev/null

**Check**

## Méthode 2 : Magic command
**Start**

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --port 21277 --host 0.0.0.0 --logdir ./run/logs

**Stop**  
No way... use bash method

## Methode 3  : Tensorboard module

**Start**

In [ ]:
import tensorboard.notebook as tsb

In [ ]:
tsb.start('--port 21277 --host 0.0.0.0 --logdir ./run/logs')

**Check**

In [ ]:
a=tsb.list()

**Stop**  
No way... use bash method

In [ ]:
!kill 214798